In [1]:
from IPython.display import display, HTML
display(
    HTML("""
<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""))


# <span style='color:red'>Ch3. 연관분석</span>

- pip install apriori

# 1. 연관분석 개요

- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관석이 어느 정도 있는지 분석
- 활용분야 : 이벤트 미리 감지, 신상품카테고리 구성
```
[조건: left-hand side : 오렌지주스] → [결과:right-hand : 와인]
- 연관분석과 관련된 지표
1. 지지도(support) : 얼마나 자주 함께 나타나는지 = 0.2
   (lhs, rhs)의 항목수 / 전체함옥수
2. 신뢰도 (confidence) : 조건이 오면 결과가 얼마나 자주 나타나는지
   (lhs → rhs)의 항목수 / lhs의 항목수 
   = 1 / 2 = 0.5
3. 항상도 (lift) : 우연히 발생한 규칙은 아닌지 확인
   (lhs → rhs의 지지도 / lhs의 지지도 * rhs 지지도) 
   = 0.2 / (0.4 * 0.6) =  0.2 / 0.24 = 0.833
   향상도 < 1 : 기대가 낮다
   향상도 > 1 : 기대가 높다
```


# 2. 연관분석 구현

In [5]:
import csv

with open('data/cf_basket.csv','r',encoding='utf-8') as f:
    csvdata = csv.reader(f)
    # print(list(csvdata))
    transaction = list(csvdata)

transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [11]:
from apyori import apriori

rules = apriori(
    transaction, # 2차원 데이터이어야 됨
    min_support=0.15,
    min_confidence=0.1,
)

rules = list(rules)
len(rules)

18

In [20]:
rules[10]

RelationRecord(items=frozenset({'콜라', '소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라', '소주'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [29]:
rule = rules[10]
support = rule[1]
order_st = rule[2]

for item in order_st:
    lhs = item[0]
    rhs = item[1]
    confidence = item[2]
    lift = item[3]

    if lift > 1:
        print('lhs : {} → rhs : {} \t support : {} \t confidence : {} \t lift : {}'.format(lhs, rhs, support, round(confidence,2),round(lift,2)))


lhs : frozenset({'소주'}) → rhs : frozenset({'콜라'}) 	 support : 0.6 	 confidence : 1.0 	 lift : 1.25
lhs : frozenset({'콜라'}) → rhs : frozenset({'소주'}) 	 support : 0.6 	 confidence : 0.75 	 lift : 1.25


In [39]:
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    
    for item in order_st:
        lhs = ', '.join([ data for data in item[0]])
        rhs = ', '.join([ data for data in item[1]])
        confidence = item[2]
        lift = item[3]

        if lift > 1:
            print('{} → {} \t {} \t {} \t {}'.format(lhs, rhs, support, round(confidence,2),round(lift,2)))
            

맥주 → 콜라 	 0.4 	 1.0 	 1.25
콜라 → 맥주 	 0.4 	 0.5 	 1.25
소주 → 콜라 	 0.6 	 1.0 	 1.25
콜라 → 소주 	 0.6 	 0.75 	 1.25
콜라 → 맥주, 소주 	 0.2 	 0.25 	 1.25
맥주, 소주 → 콜라 	 0.2 	 1.0 	 1.25
맥주 → 콜라, 와인 	 0.2 	 0.5 	 1.25
콜라 → 맥주, 와인 	 0.2 	 0.25 	 1.25
맥주, 와인 → 콜라 	 0.2 	 1.0 	 1.25
콜라, 와인 → 맥주 	 0.2 	 0.5 	 1.25
소주 → 오렌지주스, 콜라 	 0.2 	 0.33 	 1.67
콜라 → 오렌지주스, 소주 	 0.2 	 0.25 	 1.25
오렌지주스, 소주 → 콜라 	 0.2 	 1.0 	 1.25
오렌지주스, 콜라 → 소주 	 0.2 	 1.0 	 1.67
콜라 → 와인, 소주 	 0.2 	 0.25 	 1.25
와인, 소주 → 콜라 	 0.2 	 1.0 	 1.25


In [ ]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
# rules_df.loc[0] = ['와인','오렌지',0.15, 0.5, 1.1] 식으로 for문내에서 데이터 주기

idx = 0


for rule in rules:
    support = rule[1]
    order_st = rule[2]
    
    for item in order_st:
        lhs = ', '.join([ data for data in item[0]])
        rhs = ', '.join([ data for data in item[1]])
        confidence = item[2]
        lift = item[3]

        if lift > 1:
            rules_df.loc[idx] = [lhs, rhs, support,round(confidence,2), round(lift,2)] 
            idx += 1
            
rules_df.sort_values(by=['향상도','신뢰도'], ascending=False)



,lhs,rhs,지지도,신뢰도,향상도
13,"오렌지주스, 콜라",소주,0.2,1.00,1.67
10,소주,"오렌지주스, 콜라",0.2,0.33,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"맥주, 소주",콜라,0.2,1.00,1.25
8,"맥주, 와인",콜라,0.2,1.00,1.25
12,"오렌지주스, 소주",콜라,0.2,1.00,1.25
15,"와인, 소주",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25


# 3. 경주/전주여행 자료 연관분석

In [68]:
import pandas as pd
from konlpy.tag import Hannanum
from mecab import MeCab

In [69]:
df = pd.read_csv('data/naver_kin.csv', sep='\t')
total_text_list = df['total_text'].to_list()
total_text_list[:2]


[' 전주  가볼만한곳 추천 받아요  추억의 7080 다양한체험 7080감성 추억여행 테마박물관 유익한시간 2 전북 전북투어패스 통합이용권 전북핫플 여러여행지 다양한체험 카페이용추가 전주여행 필수 편안하고 즐거운 날이 되시길 바라겠습니다 감사합니다 ',
 ' 전주여행 갈려고하는데요!  전주여행 을 갈려고하는데요 아는사람과 갈려고하는데 호텔은 좋은가격에 정했고 음 2박3일여행인데 얼마정도갖고가면좋을까요 그리고 맛집같은거 카페같은거 추천해주세요 안녕하세요 전주 여행 계획 중이시네요 한옥마을 근처 ']

In [81]:
analyzer = MeCab()
total_noun_list = []
select_pos = ['NNP','NNG'] # 보통명사, 고유명사
불용어 = {'여행','여행지' }

for total_text in total_text_list:
    total_noun = [ token for token, tag in analyzer.pos(total_text) if tag in select_pos and token not in 불용어 and len(token)>1]
    total_noun_list.append(total_noun)
    
total_noun_list[:2]
    


[['전주',
  '추천',
  '추억',
  '다양',
  '체험',
  '감성',
  '추억',
  '테마',
  '박물관',
  '유익',
  '시간',
  '전북',
  '전북',
  '투어',
  '패스',
  '통합',
  '용권',
  '전북',
  '다양',
  '체험',
  '카페이',
  '용추',
  '전주',
  '필수',
  '편안',
  '감사'],
 ['전주',
  '전주',
  '사람',
  '호텔',
  '가격',
  '얼마',
  '정도',
  '맛집',
  '카페',
  '추천',
  '전주',
  '계획',
  '한옥',
  '마을',
  '근처']]

In [95]:
%%time
rules = apriori(total_noun_list, min_support=0.15, min_confidence=0.2)
rules = list(rules)

import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
# rules_df.loc[0] = ['와인','오렌지',0.15, 0.5, 1.1] 식으로 for문내에서 데이터 주기

idx = 0

for rule in rules:
    support = rule[1]
    order_st = rule[2]
    
    for item in order_st:
        lhs = ', '.join([ data for data in item[0]])
        rhs = ', '.join([ data for data in item[1]])
        confidence = item[2]
        lift = item[3]

        if lift > 1:
            rules_df.loc[idx] = [lhs, rhs, support,round(confidence,2), round(lift,2)] 
            idx += 1
            
rules_df.sort_values(by=['향상도','신뢰도'], ascending=False, inplace=True)
rules_df = rules_df.reset_index(drop=True)

CPU times: user 2.97 s, sys: 60 ms, total: 3.03 s
Wall time: 3.03 s


In [96]:
pd.options.display.max_rows

60

In [98]:
rules_df.head(60)

,lhs,rhs,지지도,신뢰도,향상도
0,"전주, 제주","맛집, 여수",0.152,1.0,5.99
1,"제주, 맛집","주달, 여수",0.152,1.0,5.99
2,"제주, 주달","맛집, 여수",0.152,1.0,5.99
3,"제주, 맛집","코스, 여수",0.152,1.0,5.99
4,"코스, 제주","맛집, 여수",0.152,1.0,5.99
5,"제주, 맛집","여수, 티비",0.152,1.0,5.99
6,"제주, 티비","맛집, 여수",0.152,1.0,5.99
7,"전주, 제주","주달, 여수",0.152,1.0,5.99
8,"전주, 제주","코스, 여수",0.152,1.0,5.99
9,"전주, 제주","여수, 티비",0.152,1.0,5.99
